### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_test.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_test.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

## Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


### 1) Vectorizar documentos

In [22]:
# Generamos los índices de random
al_azar = np.random.randint(0, len(newsgroups_train.data), 5)
print(f'Los índices al azar son: {al_azar}')

Los índices al azar son: [10650   715  5034   518  4084]


In [23]:
# Generamos una función que mida la similaridad y devuelva los 5 mayores
def mayor_similaridad(idx):
  cossim = cosine_similarity(X_train[al_azar[idx]], X_train)[0]
  mostsim = np.argsort(cossim)[::-1][1:6]
  return mostsim

In [24]:
# Probamos la función con el primer documento
mayor_similaridad(0)

array([ 498, 5045, 6261, 6440, 1492])

In [25]:
# Generamos una función que imprima el análisis por texto al azar
def analisis_texto(idx):
  print(f'Documento al azar número: {al_azar[idx]}')

  # La etiqueta del texto es la siguiente
  print(f'Etiqueta del documento al azar: {newsgroups_train.target_names[y_train[al_azar[idx]]]}')

  # Imprimimos los primeros 200 caracteres del texto al azar
  print(f'Contenido del documento al azar: \n """{newsgroups_train.data[al_azar[idx]][:200]}"""\n')

  # Imprimimos los índices de los documentos similares a este documento
  print(f'Índices de los documentos similares: {mayor_similaridad(idx)}')

  # Imprimimos la etiqueta de cada documento similar y los primeros 200 caracteres de cada texto similar
  for i in range(len(al_azar)):
    print(100*'-')
    print(f'Etiqueta del documento similar {i+1}: {mayor_similaridad(idx)[i]} - {newsgroups_train.target_names[y_train[mayor_similaridad(idx)[i]]]}')
    print(f'Contenido del documento similar {i+1}: \n """{newsgroups_train.data[mayor_similaridad(idx)[i]][:200]}"""\n')


In [26]:
# Vemos el análisis de los textos
for texto in range(len(al_azar)):
  analisis_texto(texto)
  print(100*'#')

Documento al azar número: 10650
Etiqueta del documento al azar: comp.os.ms-windows.misc
Contenido del documento al azar: 
 """---------- cut here ---------- part 02/03
M_XN.GGHOL*(3IZ!02'C'"YM=*][*&WT%S;)5:&$V8A= K/X@2$F[(J )CABC
M=8H#9!C@^.0%CF]P[  )'._@V/$5S@ ?'#NW61T@A&-G1/H#C'!\0)T7(*^Q
M._[3L4,X=K08CH]P[$N>7<*Q"SKV"<>N_"""

Índices de los documentos similares: [ 498 5045 6261 6440 1492]
----------------------------------------------------------------------------------------------------
Etiqueta del documento similar 1: 498 - comp.os.ms-windows.misc
Contenido del documento similar 1: 
 """---------- cut here ---------- part 01/01
begin 644 1260wn31.exe
M35KO 1D    & -$,__\@ P $     ?#_'@     !0V]P>7)I9VAT(#$Y.#DM
M,3DY,"!02U=!4D4@26YC+B!!;&P@4FEG:'1S(%)E<V5R=F5D+@T*        
M        _""""

----------------------------------------------------------------------------------------------------
Etiqueta del documento similar 2: 5045 - comp.os.ms-windows.misc
Contenido del documento simil

- En la mayoría de los casos, se entiende por contexto que la similaridad de coseno es una buena herramienta para entender qué tan cerca está el contexto de lo que se habla.
- Existe un caso en que tema del texto original al azar es de religión, pero el tema es política.

### 2) Modelos de Clasificación Naïve-Bayes

In [27]:
# Generamos una función que tome un vectorizador y un clasificador y retorne un F1 Score
def f1_vectoriz(vectorizador=TfidfVectorizer(), clasificador=MultinomialNB()):
  # Train
  X_train = vectorizador.fit_transform(newsgroups_train.data)
  y_train = newsgroups_train.target

  # Test
  X_test = vectorizador.transform(newsgroups_test.data)
  y_test = newsgroups_test.target

  # Modelo de clasificación
  clasificador = MultinomialNB()
  clasificador.fit(X_train, y_train)

  # Prediction
  y_pred = clasificador.predict(X_test)

  # F1_Score
  return f1_score(y_test, y_pred, average='macro')

In [28]:
# Probamos la función con el modelo default
f1_vectoriz()

0.5854345727938506

#### 2.1) Vectorizador: parámetros

In [29]:
# Planteamos un vectorizer con 50000 features, menor que el vocabulario inicial 101631
vect_1 = TfidfVectorizer(max_features=50000)

# Generamos el F1 Score
f1_vectoriz(vectorizador=vect_1)

0.593410793017284

In [30]:
# Planteamos un vectorizer con 20000 features, menor que el vocabulario inicial 101631
vect_2 = TfidfVectorizer(max_features=20000)

# Generamos el F1 Score
f1_vectoriz(vectorizador=vect_2)

0.6078865750826256

- Disminuyendo la cantidad de features, aumentamos el valor de F1 Score. Reducir la dimensionalidad puede mejorar la eficiencia y evitar el sobreajuste.

In [31]:
# Modifcar a False el valor de use_idf
vect_3 = TfidfVectorizer(use_idf=False)

# Generamos el F1 Score
f1_vectoriz(vectorizador=vect_3)

0.4715859558342732

- Si quitamos el use_idf vemos que el F1 score baja. La ponderación IDF puede mejorar la precisión al dar más importancia a las palabras que son menos comunes en el corpus.

In [32]:
# Modificamos el analyzer
vect_4 = TfidfVectorizer(analyzer='char')

# Generamos el F1 Score
f1_vectoriz(vectorizador=vect_4)

0.11828114898439623

- En caso de dividirlo en caracteres me disminuye el F1 score ya que no tiene mucho sentido en este caso.

In [33]:
# Usamos la siguiente lista de stop words --> fuente: https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py
# Stop words
STOP_WORDS = set(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at

back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by

call can cannot ca could

did do does doing done down due during

each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except

few fifteen fifty first five for former formerly forty four from front full
further

get give go

had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred

i if in indeed into is it its itself

keep

last latter latterly least less ll

just

made make many may me meanwhile might mine more moreover most mostly move much
must my myself

name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere

of off often on once one only onto or other others otherwise our ours ourselves
out over own

part per perhaps please put

quite

rather re really regarding

same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such

take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two

under until up unless upon us used using

various ve very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would

yet you your yours yourself yourselves
""".split()
)

contractions = ["n't", "'d", "'ll", "'m", "'re", "'s", "'ve"]
STOP_WORDS.update(contractions)

for apostrophe in ["‘", "’"]:
    for stopword in contractions:
        STOP_WORDS.add(stopword.replace("'", apostrophe))
STOP_WORDS = list(STOP_WORDS)

In [46]:
# Modificamos el vectorizador con esta lista
vect_5 = TfidfVectorizer(stop_words=STOP_WORDS)

# Generamos el F1 Score
f1_vectoriz(vectorizador=vect_5)

0.6489210209931484

- Mejora notablemente en este caso ya que eliminar palabras vacías tiene impacto positivo en la eficiencia y la precisión del modelo al enfocarse en términos más relevantes.

#### 2.2) Modelo MultinomialNB: parámetros

In [47]:
# Utilizamos la misma función f1_vectoriz pero
# modificamos parámetros del modelo MultinomialNB
# Usamos como parámetros del vectorizador la constante STOP_WORDS: vect_5

# Modificamos alpha
alpha = 0.9
clasif_2 = MultinomialNB(alpha=alpha)

# Calculamos el f1_score
f1_vectoriz(vectorizador=vect_5, clasificador=clasif_2)


0.6489210209931484

In [48]:
# Usamos un nuevo alpha más chico
# Modificamos alpha
alpha = 0.7
clasif_3 = MultinomialNB(alpha=alpha)

# Calculamos el f1_score
f1_vectoriz(vectorizador=vect_5, clasificador=clasif_3)

0.6489210209931484

- Al bajar este parámetro, no tenemos una mejora en el score final. Esto puede ser así, al no tener un sobreajuste.

In [49]:
# Modificamos fit prior
fit_prior = False
clasif_4 = MultinomialNB(fit_prior=fit_prior)

# Calculamos el f1_score
f1_vectoriz(vectorizador=vect_5, clasificador=clasif_4)

0.6489210209931484

- No tenemos un cambio visible.

#### 2.3) Modelo ComplementNB

In [50]:
# Usamos el mismo vectorizador y modificamos el clasificador ComplementNB
# Usamos primero el vectorizador original
clasif_5 = ComplementNB()

# Calculamos el f1_score
f1_vectoriz(clasificador=clasif_5)


0.5854345727938506

In [51]:
# Usamos el vectorizador 5
clasif_6 = ComplementNB()

# Calculamos el f1_score
f1_vectoriz(vectorizador=vect_5, clasificador=clasif_6)

0.6489210209931484

- Si bien, a priori, los valores de score de este modelo
no son mejores a los del MultinomialNB (deberían por ser ComplementNB un corrector de las suposiciones del MultinomialNB), puede deberser al haber unsado un vectorizador de muy buena performance.

### 3) Matriz Término-Documento

In [151]:
# Planteamos la transposición de las palabras vectorizadas
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_train = X_train.T

In [152]:
# Vemos entonces el shape de la T
X_train.shape

(101631, 11314)

In [153]:
# Generamos los índices de random
al_azar = np.random.randint(0, X_train.shape[0], 5)
print(f'Los índices al azar son: {al_azar}')

Los índices al azar son: [30063 78091 70840 21945 91625]


In [154]:
# Es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [155]:
# Genero una función que busque la palabra dentro del vocabulario
def buscar_palabra(idx):
  vocab = idx2word
  for indice, palabra in vocab.items():
    if indice == idx:
      return palabra
  return None

In [156]:
# Generamos una función que imprima el análisis por palabra al azar
def analisis_palabra(idx):
  print(f'Palabra al azar número: {al_azar[idx]}')

  # Imprimimos la palabra buscada
  print(f'La palabra es: {buscar_palabra(al_azar[idx])}')

  # Imprimimos los índices de las palabras similares a esta
  print(f'Índices de las Palabras similares: {mayor_similaridad(idx)}')

  # Imprimimos las palabras similares
  for i in range(len(al_azar)):
    print(f'Palabras similares: {buscar_palabra(mayor_similaridad(idx)[i])}')


In [157]:
# Vemos el análisis de las palabras y sus similares
for palabra in range(len(al_azar)):
  analisis_palabra(palabra)
  print(100*'#')

Palabra al azar número: 30063
La palabra es: cotton
Índices de las Palabras similares: [38309 60655 97419 52486 71765]
Palabras similares: evicted
Palabras similares: mechanization
Palabras similares: wrath_
Palabras similares: joad
Palabras similares: plantations
####################################################################################################
Palabra al azar número: 78091
La palabra es: reversal
Índices de las Palabras similares: [15332 14889 49234 98499 78096]
Palabras similares: _new_
Palabras similares: _bloody_hell_no_
Palabras similares: imnsho
Palabras similares: xians
Palabras similares: reversible
####################################################################################################
Palabra al azar número: 70840
La palabra es: personailty
Índices de las Palabras similares: [63169 26767 70840 43876 82002]
Palabras similares: morte
Palabras similares: charater
Palabras similares: personailty
Palabras similares: goffer
Palabras similares: shakesp

In [159]:
# Planteamos lo mismo con un número menor de features y analizamos nuevamente
tfidfvect_10000 = TfidfVectorizer(max_features=10000)
X_train = tfidfvect_10000.fit_transform(newsgroups_train.data)
X_train = X_train.T

# Vemos entonces el shape de la T
print(f'El shape es: {X_train.shape}\n')

# Generamos los índices de random
al_azar = np.random.randint(0, X_train.shape[0], 5)
print(f'Los índices al azar son: {al_azar}')

# Es muy útil tener el diccionario opuesto que va de índices a términos
idx2word_10000 = {v: k for k,v in tfidfvect_10000.vocabulary_.items()}

# Genero una función que busque la palabra dentro del vocabulario
def buscar_palabra(idx):
  vocab = idx2word_10000
  for indice, palabra in vocab.items():
    if indice == idx:
      return palabra
  return None

# Vemos el análisis de las palabras y sus similares
for palabra in range(len(al_azar)):
  analisis_palabra(palabra)
  print(100*'#')

El shape es: (10000, 11314)

Los índices al azar son: [5445 4525 5702 1548 3308]
Palabra al azar número: 5445
La palabra es: linked
Índices de las Palabras similares: [9558 2016  997 4125 1451]
Palabras similares: vpic46
Palabras similares: cfg
Palabras similares: allocation
Palabras similares: gfx
Palabras similares: barrier
####################################################################################################
Palabra al azar número: 4525
La palabra es: hope
Índices de las Palabras similares: [4425 9026 8917 8919 9949]
Palabras similares: helps
Palabras similares: to
Palabras similares: that
Palabras similares: the
Palabras similares: you
####################################################################################################
Palabra al azar número: 5702
La palabra es: master
Índices de las Palabras similares: [8264  807 2729 1720 5103]
Palabras similares: slave
Palabras similares: accord
Palabras similares: cylinder
Palabras similares: brake
Palabras similar

- Si vamos con el número de features original (vocabulario 101631) muchas de las palabras dentro del diccionario son palabras que no forman parte del vocabulario Inglés persé, sino palabras que quedaron dentro del vocabulario del texto como "malos tokens".
- Incluso, bajando el número de features, tenemos estas palabras pero en menor proporción.
- Al ir con 10000 palabras, si bien el resultado de las similares son palabras del inglés, no existe mucha similaridad contextual entre ellas.